## Import all the necessary library

In [65]:
import pandas as pd
import numpy as np

<div class="alert alert-info" role="alert">
  <span style="font-size: larger;"><strong>1. Read all the csv files into DataFrames</strong></span>
</div>

In [66]:
df_customers = pd.read_csv("Generated_Data_Uniformed\customers.csv")
df_shirts = pd.read_csv("Generated_Data_Uniformed\shirts.csv")
df_invoice_details = pd.read_csv("Generated_Data_Uniformed\invoice_details.csv")
df_invoices = pd.read_csv("Generated_Data_Uniformed\invoices.csv")
df_demands = pd.read_csv("Generated_Data_Uniformed\demands.csv")
df_imports = pd.read_csv("Generated_Data_Uniformed\imports.csv")
df_shops = pd.read_csv("Generated_Data_Uniformed\shops.csv")
df_import_details = pd.read_csv("Generated_Data_Uniformed\import_details.csv")

<div class="alert alert-info" role="alert">
  <span style="font-size: larger;"><strong>2. Calculate the percentage distribution of each category of shirt</strong></span>
</div>

In [67]:
# merge df_invoices and df_invoice_details and group by ShirtID for each unique shirt color and size
df = pd.merge(df_invoices, df_invoice_details, on="InvoiceID")
df = df.groupby('ShirtID').agg(Count=("Quantity", "sum")).reset_index()

# store necessary data into necessary variable
shirt_sold_each_category = df["Count"]
total_shirt_sold = df["Count"].sum()

# calculate the percentages of each category
percentage_for_each_category = shirt_sold_each_category/total_shirt_sold
percentage_for_each_category

0     0.024789
1     0.025660
2     0.027575
3     0.019880
4     0.020402
5     0.024337
6     0.023466
7     0.020577
8     0.021865
9     0.019741
10    0.022317
11    0.020402
12    0.021099
13    0.023536
14    0.024406
15    0.022526
16    0.020820
17    0.019079
18    0.019671
19    0.022213
20    0.020194
21    0.022213
22    0.021726
23    0.022248
24    0.020333
25    0.021551
26    0.022213
27    0.022631
28    0.018105
29    0.020820
30    0.021099
31    0.021969
32    0.023501
33    0.024197
34    0.023397
35    0.022805
36    0.024441
37    0.021691
38    0.023327
39    0.023188
40    0.021760
41    0.023292
42    0.021482
43    0.023153
44    0.024302
Name: Count, dtype: float64

<div class="alert alert-info" role="alert">
  <span style="font-size: larger;"><strong>3. Calculate the recommeded stocking quantity by using the safety stock formula</strong></span>
</div>

In [68]:
# store lead_times and sales into varible, the lead_times are divided by 30 due to fact it's in term of months in our case
lead_times = (df_imports["AcutalWaitingDays"] - df_imports["ExpectedWaitingDays"])/30
number_of_month_to_stock_for = 6
sales = df_demands["QuantitySold"]

# using the safety stock formula and normal stock
safety_stock = ((np.max(lead_times) * np.max(sales)) - (np.average(lead_times) * np.average(sales))) * number_of_month_to_stock_for
stock = np.average(sales) * number_of_month_to_stock_for
stock = np.round(stock)

# apply the percentage distribution percentages of each category over the amount from the safety stock
quantity_for_each_category = percentage_for_each_category * stock
backup_quantity_for_each_category = percentage_for_each_category * safety_stock
print("Amount to stock:\n", list(quantity_for_each_category))
print("Amount to stock for backup:\n", list(backup_quantity_for_each_category))

Amount to stock:
 [94.94324907736231, 98.2769305758652, 105.61102987257155, 76.141285425806, 78.14149432490774, 93.20973469814079, 89.87605319963791, 78.80823062460831, 83.74207924239259, 75.60789638604554, 85.4755936216141, 78.14149432490774, 80.80843952371005, 90.14274771951813, 93.47642921802104, 86.27567718125479, 79.74166144418913, 73.07429844718334, 75.3412018661653, 85.07555184179375, 77.34141076526704, 85.07555184179375, 83.20869020263213, 85.20889910173386, 77.87479980502751, 82.54195390293154, 85.07555184179375, 86.67571896107513, 69.34057516886011, 79.74166144418913, 80.80843952371005, 84.14212102221293, 90.00940045957803, 92.67634565838034, 89.60935867975768, 87.34245526077572, 93.60977647796113, 83.07534294269202, 89.34266415987746, 88.80927512011698, 83.34203746257224, 89.20931689993733, 82.27525938305132, 88.67592786017687, 93.07638743820068]
Amount to stock for backup:
 [10.795848942738436, 11.174916672469418, 12.008865677877585, 8.657906947055682, 8.885347584894275, 10

<div class="alert alert-info" role="alert">
  <span style="font-size: larger;"><strong>4. Function to calculate the quantity for each category and round the result</strong></span>
</div>

In [69]:
def round_array_to_interger(array):
    # store the orginal quantity_for_each_category into a numpy array and the original sum of the array
    original_array = array.copy()
    original_sum = np.sum(original_array)

    # find the fractional part of each number in the numpy array
    fractional_parts = original_array - np.floor(original_array)

    # round the array and find the sum of the rounded array
    rounded_array = np.round(original_array).astype(int)
    rounded_sum = np.sum(rounded_array)

    # calculate the difference between orginal sum and the sum from the rounded array
    difference = original_sum - rounded_sum

    # if difference is bigger than 0 distribution the difference to 
    # the biggest numbers that had been rounded down
    if difference > 0:
        sorted_indices = np.argsort(fractional_parts)[::-1]
        for idx in sorted_indices:
            if fractional_parts[idx] < 0.5:
                rounded_array[idx] += 1
                difference -= 1
                if difference == 0:
                    break
                    
    # if diffrence is smaller than 0 distribution the difference to
    # the smallest numbers that had been rounded up
    elif difference < 0:
        sorted_indices = np.argsort(fractional_parts)
        for idx in sorted_indices:
            if fractional_parts[idx] >= 0.5:
                rounded_array[idx] -= 1
                difference += 1
                if difference == 0:
                    break
                    
    return rounded_array

In [70]:
recommned_quantity_to_import = round_array_to_interger(quantity_for_each_category)
backup_quantity_to_import = round_array_to_interger(backup_quantity_for_each_category)
print("Recommended Quantity To Import", list(recommned_quantity_to_import))
print("Backup_Quantity_Import: ", list(backup_quantity_to_import))

Recommended Quantity To Import [95, 99, 106, 77, 79, 94, 90, 79, 84, 76, 86, 79, 81, 91, 94, 87, 80, 74, 76, 86, 78, 86, 84, 86, 78, 83, 86, 87, 70, 80, 81, 85, 91, 93, 90, 88, 94, 84, 90, 89, 84, 90, 83, 89, 94]
Backup_Quantity_Import:  [10, 11, 12, 8, 8, 10, 10, 8, 9, 8, 9, 8, 9, 10, 10, 9, 9, 8, 8, 9, 8, 9, 9, 9, 8, 9, 9, 9, 7, 9, 9, 9, 10, 10, 10, 9, 10, 9, 10, 10, 9, 10, 9, 10, 10]


<div class="alert alert-info" role="alert">
  <span style="font-size: larger;"><strong>5. Merge other attributes of each ShirtID with the Recommended_Quantity_To_Import and Optinal_Backup_Quantity_To_Import</strong></span>
</div>

In [71]:
# create a dictionary storing ShirtID and the Recommended_Quantity_To_Import for each ShirtID
data_recommended_quantity = {
    "ShirtID" : [i+1 for i in range(45)],
    "Recommended_Quantity_To_Import" : recommned_quantity_to_import,
    "Optinal_Backup_Quantity_To_Import" : backup_quantity_to_import
}

# create the DataFrame from the dictionary
df_recommended_quantity = pd.DataFrame(data_recommended_quantity)

# merge the df_shirts and df_recommended_quantity so that we get the ShirtID, other attributes
# and Recommended_Quantity_To_Import for each ShirtID in one DataFrame
df = pd.merge(df_shirts, df_recommended_quantity, on="ShirtID")
df = df.drop("StockQty", axis=1)
df.sort_values(by=["Color", "Size"], inplace=True)
df.reset_index(drop=True, inplace=True)
df.to_csv("RecommendedStockingQuantity.csv", index=False)
df

,ShirtID,Color,Size,UnitPrice,Discount,ShopID,Recommended_Quantity_To_Import,Optinal_Backup_Quantity_To_Import
0,12,black,L,11,0.4,2,79,8
1,16,black,M,6,0.2,4,87,9
2,14,black,S,6,0.2,1,91,10
3,22,black,XL,5,0.4,3,86,9
4,43,black,XXL,4,0.3,5,83,9
5,40,blue,L,10,0.3,4,89,10
6,1,blue,M,7,0.3,1,95,10
7,28,blue,S,9,0.2,3,87,9
8,7,blue,XL,7,0.1,5,90,10
9,30,blue,XXL,9,0.4,5,80,9
